State of Charge (SoC) Estimation Methods: Overview and Comparison
In battery management systems (BMS), State of Charge (SoC) is like a fuel gauge for batteries — it tells how much usable energy remains.
However, you can't measure SoC directly, like you can measure voltage or current. Instead, we estimate it.

There are five major classes of SoC estimation methods:
 
1) Direct Measurement (OCV), 2) Coulomb Counting (Ah-Integration), 3) Model-Based Estimation 4) Adaptive / Observer-based and 5) Data-Driven / AI-based

In [1]:
def ah_integration(soc_initial, current, time_step, capacity):
    """
    Estimate SoC using Ah integration.
    soc_initial: Initial SoC (fraction, 0 to 1)
    current: Current in A (positive for charge, negative for discharge)
    time_step: Time interval in hours
    capacity: Battery capacity in Ah
    """
    charge_change = current * time_step  # Ah added or removed
    soc_change = charge_change / capacity
    soc_new = soc_initial + soc_change
    return max(0, min(1, soc_new))  # Clamp between 0 and 1

# Example usage
soc_0 = 0.8  # 80%
current = -5 # 5A discharge
time_step = 2  # 2 hours
capacity = 100  # 100 Ah

soc = ah_integration(soc_0, current, time_step, capacity)
print(f"New SoC: {soc * 100:.1f}%")

New SoC: 70.0%


In [2]:
soc_0 = 0.7  # 70%
current = -10 # 10A discharge
time_step = 1  # 1 hours
capacity = 100  # 100 Ah

soc = ah_integration(soc_0, current, time_step, capacity)
print(f"New SoC: {soc * 100:.1f}%")

New SoC: 60.0%


In [3]:
def ocv_to_soc(voltage, ocv_table):
    """
    Estimate SoC from OCV using interpolation.
    voltage: Measured open-circuit voltage (V)
    ocv_table: Dictionary of {voltage: soc} pairs
    """
    voltages = sorted(ocv_table.keys())
    socs = [ocv_table[v] for v in voltages]
    
    if voltage <= voltages[0]:
        return socs[0]
    if voltage >= voltages[-1]:
        return socs[-1]
    
    # Linear interpolation
    for i in range(len(voltages) - 1):
        if voltages[i] <= voltage < voltages[i + 1]:
            v0, v1 = voltages[i], voltages[i + 1]
            s0, s1 = socs[i], socs[i + 1]
            soc = s0 + (s1 - s0) * (voltage - v0) / (v1 - v0)
            return soc

# Example OCV-SoC table
ocv_table = {3.6: 0.2, 3.8: 0.5, 4.0: 0.8}
voltage = 3.8

soc = ocv_to_soc(voltage, ocv_table)
print(f"SoC: {soc * 100:.1f}%")

SoC: 50.0%


In [4]:
# Enhanced Version (Optional)
# Here’s a polished version with some improvements:  20 Mar25
def ocv_to_soc(voltage, ocv_table):
    """
    Estimate SoC from OCV using interpolation.
    voltage: Measured open-circuit voltage (V)
    ocv_table: Dictionary of {voltage: soc} pairs
    """
    if not ocv_table:
        raise ValueError("OCV table cannot be empty")
    
    voltages = sorted(ocv_table.keys())
    socs = [ocv_table[v] for v in voltages]
    
    if voltage <= voltages[0]:
        return socs[0]
    if voltage >= voltages[-1]:
        return socs[-1]
    
    for i in range(len(voltages) - 1):
        if voltages[i] <= voltage < voltages[i + 1]:
            v0, v1 = voltages[i], voltages[i + 1]
            s0, s1 = socs[i], socs[i + 1]
            soc = s0 + (s1 - s0) * (voltage - v0) / (v1 - v0)
            return soc
    
    raise ValueError("Interpolation failed unexpectedly")

# Example with a more realistic LiFePO4-like table
ocv_table = {2.5: 0.0, 3.0: 0.1, 3.2: 0.4, 3.3: 0.9, 3.6: 1.0}
voltage = 3.25

soc = ocv_to_soc(voltage, ocv_table)
print(f"SoC: {soc * 100:.1f}%")  # Interpolates between 3.2 (40%) and 3.3 (90%)

SoC: 65.0%
